# Simulación ADS frente VC



#1. Simulador de vehículos de conducción autónoma



In [442]:
import numpy as np
import pandas as pd

In [443]:
## En primer lugar creemos una función que nos diga cómo evoluciona el vehículo en función de los parámetros (posición velocidad y aceleración).

def posicion_coche(x_a, v_a, a_a, inf_t):
  x = x_a + v_a * inf_t + (a_a/2)* inf_t ** 2
  v = v_a + a_a * inf_t
  a = a_a
  return (x,v,a)

## Ahora podemos calcular las trayectorias de una manera sencilla (teniendo siempre en cuenta la posición inicial y recorriendo t) o
## Podemos calcularlas con una función recursiva que tenga en cuenta el infinitesimal de oroden t


## Modelización del vehículo autónomo

Tenemos aquí una función que modela el comportamiento del vehículo autónomo. Esta tiene en cuenta que si el vehículo autónomo alcanza la velocidad límite de la vía, su aceleración cambia a 0 y por lo tanto se mantiene en esta.

t_ini y t_max son los valores de tiempo en los que se mueve nuestra función.

In [444]:
## Aquí ya está reflejado que el coche pueda ir más lento en el carril de incorporacíon, lo que no podrá es decelerar en dicho carril

def calcular_trayectoria_autonomo(x_a, v_a, a_a, t_ini, t_max, v_min, v_max, inf_t = 0.01):
  resultados = [(x_a, v_a, a_a, t_ini)]
  while t_ini < t_max:

    if v_a >= v_max:
      v_a = v_max
      a_a = 0
    elif (v_a <= v_min) and (a_a < 0):
      v_a = v_min
      a_a = 0

    x_a1, v_a1, a_a1 = posicion_coche(x_a, v_a, a_a, inf_t)

    t_ini += inf_t
    resultados.append((x_a1, v_a1, a_a1, t_ini))

    x_a, v_a, a_a = x_a1, v_a1, a_a1

  return resultados



In [445]:
## Probemos la función anterior:

x_a_inicial = 0
v_a_inicial = 3
a_a_inicial = -1
t_ini = 0
t_max = 20
v_min = 2
v_max = 5

trayectoria = calcular_trayectoria_autonomo(x_a_inicial, v_a_inicial, a_a_inicial, t_ini, t_max, v_min, v_max)


Con esto ya tenemos nuestra función que simula al vehículo autónomo. Creemos la función que simula al vehículo humano

## Modelización del vehículo humano

A diferencia del autónomo, en este no vamos a tener en cuenta la restricción de velocidad, pues se entiende que un humano podría tomar la decisión de superar dicho límite.

In [446]:
def calcular_trayectoria_humano(x_h, v_h, a_h, t_ini, t_max, v_min, v_max, agresivo = False, inf_t = 0.01):
  """
  El parámetro agresivo debe tomar el valor True o False refiriéndose a si el conductor del vehículo conduce de forma agresiva o no
  """
  if agresivo:
    v_max = v_max * 1.2 # Si especificamos que el conductor es agresivo, nos encontraremos que puede rebasar la velocidad máxima en un 20%

  resultados = [(x_h, v_h, a_h, t_ini)]

  while t_ini < t_max:

    if v_h >= v_max:
      v_h = v_max
      a_h = 0
    elif (v_h <= v_min) and (a_h < 0): # Tenemos en cuenta también que el conductor nunca va a bajar la velocidad mínima de la vía
      v_h = v_min
      a_h = 0

    x_h1, v_h1, a_h1 = posicion_coche(x_h, v_h, a_h, inf_t)

    t_ini += inf_t
    resultados.append((x_h1, v_h1, a_h1, t_ini))

    x_h, v_h, a_h = x_h1, v_h1, a_h1

  return resultados

In [447]:
## Probemos la función anterior:

x_h_inicial = 0
v_h_inicial = 22
a_h_inicial = 0.5
t_ini = 0
t_max = 20
v_min = 12.5
v_max = 25
agresivo = True

trayectoria = calcular_trayectoria_humano(x_h_inicial, v_h_inicial, a_h_inicial, t_ini, t_max, v_min, v_max, agresivo)

for i in range(10):
  print(trayectoria[i])

(0, 22, 0.5, 0)
(0.220025, 22.005, 0.5, 0.01)
(0.4401, 22.009999999999998, 0.5, 0.02)
(0.6602250000000001, 22.014999999999997, 0.5, 0.03)
(0.8804000000000001, 22.019999999999996, 0.5, 0.04)
(1.100625, 22.024999999999995, 0.5, 0.05)
(1.3209, 22.029999999999994, 0.5, 0.060000000000000005)
(1.5412249999999998, 22.034999999999993, 0.5, 0.07)
(1.7615999999999996, 22.039999999999992, 0.5, 0.08)
(1.9820249999999995, 22.04499999999999, 0.5, 0.09)


## Cálculo de las restricciones

Calculemos ahora cúando no se cumplen las restricciones de nuestro problema, es decir, calculemos el tiempo en el que la distancia entre el vehículo autónomo y el vehículo con conductor humano están a menos de 5 metros de distancia. Además, calculemos esto también mientras el vehículo autónomo está en el carril de incorporación. Es decir, no tiene sentido que el vehículo autónomo pueda realizar la maniobra si se le termina el carril.



En primer lugar creemos alguna función adicional que necesitaremos más adelante, en concreto una que cuente valores consecutivos en una lista.

In [448]:
## Definamos una función que cuente los True's consecutivos en una lista de pyhton

def contar_consecutivos(lista, num_consecutivos):
    contador_consecutivos = 0
    secuencias = []
    indices_inicio = []

    for i, valor in enumerate(lista):
        if valor:
            if contador_consecutivos == 0:
                inicio = i  # Marca la posición de inicio de una nueva secuencia de consecutivos
            contador_consecutivos += 1
        else:
            if contador_consecutivos > num_consecutivos:
                secuencias.append(contador_consecutivos)
                indices_inicio.append(inicio)
            contador_consecutivos = 0

    # Verificar al final de la lista si los consecutivos son mayores al número que buscamos
    if contador_consecutivos > num_consecutivos:
        secuencias.append(contador_consecutivos)
        indices_inicio.append(inicio)

    # Si no hay, la lista secuencia y la lista indices_inicio serán listas vacías

    return secuencias, indices_inicio

En segundo lugar creemos un bucle que nos ejecute las trayectorias calculadas anterioremente anteriores y nos diga si se puede realizar la maniobra (es decir, si en 5 segundos hay 5 metros de distancia horizontal entre los 2 coches en todo momento)


In [449]:
## Esto sería lo que ejecutaría el coche autónomo para poder ver si la maniobra se puede o no realizar

# Para ello el coche autónomo necesita saber (que lo calculará con sus sensores) la posición, velocidad y aceleración del vehículo con conductor humano
# Evidentemente necesitará saber su posición.



def maniobra_factible(x_a, v_a, a_a, x_h, v_h, a_h, x_final, v_min, v_max, agresivo = False, d_seguridad = 5, t_maniobra_min = 5, inf_t = 0.01):
  """
  Esta función nos devuelve si la maniobra de incorporación es factible en función de los siguientes parámetros

  x_a: posición inicial vehículo autónomo
  v_a: velocidad inicial vehículo autónomo
  a_a: aceleración inicial vehículo autónomo

  x_h: posición inicial vehículo con conductor
  v_h: velocidad inicial vehículo con conductor
  a_h: aceleración inicial vehículo con conductor

  x_final: posición final de la vía de incorporación

  v_min: velocidad mínima de la vía
  v_max: velocidad máxima de la vía

  agresivo: Booleano que nos indica si el conductor del vehículo con conductor tiene una conducción agresiva
  d_seguridad: Distancia que tendrá en cuenta la función para considerar si la maniobra es segura
  t_maniobra_min: tiempo que se tarda en realizar la maniobra
  inf_t: infinitesimal de orden t a considerar
  """

  # Creemos un bucle cuya condicicón sea que no se nos finalize el carril de aceleración
  # (para ello restemos la distancia del vehículo autónomo menos la distancia del punto final del carril)

  if agresivo:
    v_max_h = v_max * 1.2 # Si especificamos que el conductor es agresivo, nos encontraremos que puede rebasar la velocidad máxima en un 20%
  else:
    v_max_h = v_max

  t_ini = 0 # Definamos el tiempo inicial
  lista_valores = [[t_ini, x_a, v_a, a_a, x_h, v_h, a_h]] # Definamos nuestra lista de resultados y le añadimos los datos iniciales

  while x_final - x_a > 0:

    # Calculemos las posiciones de nuestro vehículo autónomo (tendremos en cuenta velocidad máxima y velocidad mínima)
    if v_a >= v_max:
      v_a = v_max
      a_a = 0
    elif (v_a <= v_min) and (a_a < 0):
      v_a = v_min
      a_a = 0

    x_a1, v_a1, a_a1 = posicion_coche(x_a, v_a, a_a, inf_t)

    # Calculemos las posiciones de nuestro vehículo humano
    if v_h >= v_max_h:
      v_h = v_max_h
      a_h = 0
    elif (v_h <= v_min) and (a_h < 0): # Tenemos en cuenta también que el conductor nunca va a bajar la velocidad mínima de la vía
      v_h = v_min
      a_h = 0

    x_h1, v_h1, a_h1 = posicion_coche(x_h, v_h, a_h, inf_t)

    # Sumemos el infinitesimal de orden t a nuestro tiempo inicial
    t_ini += inf_t

    # Añadamos los resultados a nuestra lista de valores y actualicemos estos para que pueda continuar el bucle
    lista_valores.append([t_ini, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1])

    x_a, v_a, a_a, x_h, v_h, a_h = x_a1, v_a1, a_a1, x_h1, v_h1, a_h1


  ## Ahora tenemos que ver las distancias entre los dos coches
  lista_valores_np = np.array(lista_valores)

  # En primer lugar calculamos el valor absoluto de las diferencias
  diferencia = abs(lista_valores_np[:,1] - lista_valores_np[:,4])
  diferencia = diferencia.reshape(-1, 1)

  # Añadimos la columna a nuestra matriz
  lista_valores_np = np.hstack((lista_valores_np, diferencia))

  # Ahora veamos qué distancias son superiores a la distancia de seguridad (5 (metros))
  superiores = diferencia > d_seguridad
  superiores.reshape(-1,1)
  lista_valores_np = np.hstack((lista_valores_np, superiores))

  ## Ahora tenemos que ver que esa condición se cumpla durante el t_maniobra_min
  # Para ello tenemos que tener en cuenta de cuánto es nuestro infinitesimal de orden t para
  # calcular el número de True's que tienen que ir seguidos en la columna superiores

  # Calculemos cuantos infinitesimales con True tienen que ir seguidos en funcion de nuestro t_maniobra_min
  num_seguidos = t_maniobra_min/inf_t

  ## Usemos la función creada anteriormente para contar los valores consecutivos de nuestra lista superiores
  secuencias, indices_inicio_secuencia = contar_consecutivos(superiores, num_seguidos)


  if secuencias != []:
    # Obtengamos las posiciones y los tiempos en los que el vehículo autónomo puede comenzar la maniobra, así como los
    # segundos de los que dispondrá para realizar dicha maniobra
    resultados = [['Maniobra', "t_inicio_maniobra", "x_a", "v_a", "a_a", "x_h", "v_h", "a_h", "dist_coches", "bool", "Tiempo para la maniobra", "Velocidad ADS final maniobra"]]

    for indice in range(len(secuencias)):
      tiempo_disp_maniobra = secuencias[indice]*inf_t
      i_ini_maniobra = indices_inicio_secuencia[indice]
      velocidad_a_final_maniobra = lista_valores_np[i_ini_maniobra + int(num_seguidos)].tolist()[2] # Sumamos num_seguidos para saber la velocidad final de la maniobra
      valores_vehiculos = lista_valores_np[i_ini_maniobra].tolist()
      resultado_maniobra = [True] + valores_vehiculos + [tiempo_disp_maniobra] + [velocidad_a_final_maniobra]
      resultados.append(resultado_maniobra)

    #print("Se puede realizar la maniobra")
    return pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0]))
    ### return("Se puede realizar la maniobra", pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))
    #return(pd.DataFrame(lista_valores_np))

  else:
    resultados = [['Maniobra']]
    ### return("No se puede realizar la maniobra")
    resultados.append([False])
    return(pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))



Probemos si funciona nuestra función con un par de ejemplos:

### Ejemplo 1: No se puede realizar la maniobra

In [450]:
## EJEMPLO 1

x_a0 = 15
v_a0 = 20
a_a0 = 0.25

x_h0 = 0
v_h0 = 22.5
a_h0 = 0.15

v_min = 12.5
v_max = 25
x_fin_carril = 200

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01


maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_max = v_max, v_min = v_min)



,Maniobra
0,False


Aquí tenemos un ejemplo donde no se puede realizar el cambio de carril


### Ejemplo 2: Se puede realizar la maniobra

In [451]:
## Ejemplo 2

x_a0 = 15
v_a0 = 20
a_a0 = 0.3

x_h0 = 0
v_h0 = 22.5
a_h0 = 0

v_min = 12.5
v_max = 25
x_fin_carril = 200

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01


maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max)

,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,0.0,15.0,20.0,0.3,0.0,22.5,0.0,15.0,1.0,6.67,21.5


Aquí tenemos un ejemplo donde se puede realizar el cambio de carril en un punto

### Ejemplo 3: Se puede realizar la maniobra en dos momentos diferentes

In [452]:
## Ejemplo 3

x_a0 = 0
v_a0 = 22.5
a_a0 = 0.25

x_h0 = 15
v_h0 = 22.5
a_h0 = 0


v_min = 12.5
v_max = 25
x_fin_carril = 500

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01


maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max)



,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,0.0,0.0,22.5,0.25,15.0,22.5,0.0,15.0,1.0,8.95,23.75
1,1.0,13.0,312.5,25.0,0.00,307.5,22.5,0.0,5.0,1.0,7.51,25.00


In [453]:
## Ejemplo 4  Con agresividad = True

x_a0 = 0
v_a0 = 22.5
a_a0 = 0.25

x_h0 = 0
v_h0 = 25
a_h0 = 0.1


v_min = 12.5
v_max = 25
x_fin_carril = 500
agresividad = True

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01


maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max, agresivo = agresividad)


,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,2.14,48.72245,23.035,0.25,53.72898,25.214,0.1,5.00653,1.0,18.37,24.285


Aquí tenemos un ejemplo donde se puede realizar el cambio de carril en dos puntos diferentes

#2. Cálculo de la maniobra con varias opciones discretas de aceleración:

En esta sección se estudiará individualmente si se puede realizar la maniobra en función de la aceleración del vehículo autónomo. Tendremos una tabla de aceleraciones posibles entre las que se calculará la posibilidad de realizar la maniobra.

Para ello, tendremos las siguientes aceleraciones:

*   No acelerar: 0 m/s^2
*   Acelerar: 0.15 m/s^2
*   Decelerar: -0.15 m/s^2
*   Acelerar bruscamente: 0.30 m/s^2
*   Decelerar bruscamente: -0.30 m/s^2

Calculemos las maniobras que puede realizar nuestro vehículo autónomo en función de su aceleración.

Para ello tengamos en cuenta una incorporación en un caso usual de vía secundaria. Tendremos por ejemplo, un vehículo conducido por un humano que circula a una velocidad de 90 km/h (misma velocidad que la velocidad máxima de la vía). Veamos también que tenemos un vehículo autónomo que se quiere incorporar a la carretera y que tiene un carril de aceleración de 200 metros.

La velocidad del coche autónomo supongamos que es 60 km/h.
Además supongamos que el coche autónomo rebasa al coche conducido por un humano 10 metros, es decir, que el coche autónomo lleva 10 metros ya en el carril de aceleración.



In [454]:
## En primer lugar definamos una lista con los posibles valores de aceleración y veamos entonces que resultados obtenemos

aceleraciones = [-0.3, -0.15, 0, 0.15, 0.3]

# Introduzcamos los datos iniciales:

x_a0 = 0
v_a0 = 23
# a_a0 es la lista

x_h0 = 0
v_h0 = 25
a_h0 = 0

v_min = 12.5
v_max = 25
x_fin_carril = 200

resultados = []

i = 0
for aceleracion in aceleraciones:
  resultados.append([maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = aceleracion, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_max = v_max, v_min = v_min)])


resultados

[[   Maniobra  t_inicio_maniobra       x_a     v_a  a_a   x_h   v_h  a_h  \
  0       1.0               2.16  48.98016  22.352 -0.3  54.0  25.0  0.0   
  
     dist_coches  bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
  0      5.01984   1.0                     7.11                        20.852  ],
 [   Maniobra  t_inicio_maniobra        x_a      v_a   a_a    x_h   v_h  a_h  \
  0       1.0               2.31  52.729792  22.6535 -0.15  57.75  25.0  0.0   
  
     dist_coches  bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
  0     5.020208   1.0                     6.66                       21.9035  ],
 [   Maniobra  t_inicio_maniobra   x_a   v_a  a_a   x_h   v_h  a_h  dist_coches  \
  0       1.0                2.5  57.5  23.0  0.0  62.5  25.0  0.0          5.0   
  
     bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
  0   1.0                     6.21                          23.0  ],
 [   Maniobra  t_inicio_maniobra     x_a    v_a   a_a

Con esto hemos probado varias aceleraciones para el mismo modelo, es decir, es lo que estudiaría el coche en caso de tener que decidir qué hacer.

# 3. Modelo 1: Implementación de teoría de la utilidad para la decisión de las maniobras

A y H toman a la vez una misma decisión para que posteriormente A vuelva a tomar otra decisión, es decir, tendríamos el siguiente esquema.



Ahora mismo lo que tenemos que hacer es obtener unas funciones de utilidad que intentaremos maximizar (o minimizar) según nos beneficie. Tendremos que tener una función de utilidad para cada uno de los vehículos, tanto para el autónomo como para el conducido para un humano.

El esquema de actuación será el siguiente. Ambos coches se ven mutuamente, entonces ambos coches toman una decisión en función a su utilidad. Posteriormente el coche autónomo vuelve a tomar otra decisión.


## Calculemos las funciones que obtengan las utilidades
Aquí entiendo que la utilidad la hay que calcular una vez. Al inicio de la maniobra?

Y tengo otro problema. Si por ejemplo se da el caso de que con una aceleración tengo 2 posibles incorporaciones, calculo las 2 utilidades y me quedo con la mejor a la hora de realizar la matriz??

In [455]:
## En primer lugar veamos que decisiones puede tomar cada jugador

decisiones_ADS = [-0.3, -0.15, 0, 0.15, 0.3, 'No realizar la maniobra']

decisiones_VH = [-0.3, -0.15, 0, 0.15, 0.3]

# Los números significan: modificar la aceleración a ese número y, en caso del ADS, realizar la maniobra.

### Calculemos la utilidad del vehículo autónomo


In [456]:
## En primer lugar creemos una función que nos calcule el valor de la utilidad una vez tenemos ya los datos que forman parte de dicha función

# Esta función la utilizaremos más abajo para poder calcular la utilidad en función de las decisiones de los jugadores

def calcular_u_a(aceleracion, tiempo_maniobra, v_final, v_max, b_a, w1, w2, w3):
  """
  aceleración es la aceleración del vehículo autónomo
  tiempo_maniobra es el tiempo total que existe para realizar la maniobra
  v_final es la velocidad que alcanza el coche autónomo una vez la maniobra ha finalizado

  b_a es el intercepto
  w1 es el peso correspondiente a la aceleración
  w2 es el peso correspondiente al tiempo extra
  w3 es el peso correspondiente a la velocidad alcanzada tras la maniobra
  """
  # En primer lugar calculamos la indicatriz de la maniobra
  if tiempo_maniobra >= 8:
    I_t_extra = 1
  else:
    I_t_extra = 0

  # En segundo lugar calculamos la indicatriz de la velocidad final
  if v_max * 0.8 <= v_final and v_final <= v_max:
    I_vel = 1
  else:
    I_vel = 0

  # Finalmente calculamos la utilidad y la devolvemos
  utilidad = b_a + w1 * abs(aceleracion) + w2 * I_t_extra + w3 * I_vel
  return utilidad


In [457]:
def calcular_utilidad_autonomo(decision_autonomo, decision_humano, x_a, v_a, x_h, v_h, x_final, v_min, v_max, b_a, w1, w2, w3, agresivo = False, d_seguridad = 5, t_maniobra_min = 5, inf_t = 0.01):
  """
  decision_autonomo se corresponde con la decisión que toma el ADS
  decisión humano se corresponde con la decisión del VH

  x_a: posición inicial vehículo autónomo
  v_a: velocidad inicial vehículo autónomo
  x_h: posición inicial vehículo con conductor
  v_h: velocidad inicial vehículo con conductor
  x_final: posición final de la vía de incorporación
  v_min: velocidad mínima de la vía
  v_max: velocidad máxima de la vía
  agresivo: Booleano que nos indica si el conductor del vehículo con conductor tiene una conducción agresiva
  d_seguridad: Distancia que tendrá en cuenta la función para considerar si la maniobra es segura
  t_maniobra_min: tiempo que se tarda en realizar la maniobra
  inf_t: infinitesimal de orden t a considerar

  b_a es el intercepto
  w1 es el peso correspondiente a la aceleración
  w2 es el peso correspondiente al tiempo extra
  w3 es el peso correspondiente a la velocidad alcanzada tras la maniobra
  """

  # En primer lugar veamos qué ocurre si la decision es no realizar la maniobra.
  if decision_autonomo == 'No realizar la maniobra':
    return 0.01

  # Ahora veamos qué ocurre cuando la decisión es realizarla
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = maniobra_factible(x_a = x_a, v_a = v_a, a_a = decision_autonomo, x_h = x_h, v_h = v_h, a_h = decision_humano, x_final = x_final, v_min = v_min, v_max = v_max, agresivo = agresivo, d_seguridad = d_seguridad, t_maniobra_min = t_maniobra_min, inf_t = inf_t)
    # Ahora comprobemos cuál de los 3 resultados tenemos
    if es_factible.iloc[0, 0] == 0: # Es decir, que no se puede realizar la maniobra
      return 0

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      aceleracion = decision_autonomo
      tiempo_maniobra = es_factible.iloc[0, 10]
      v_final = es_factible.iloc[0, 11]

      utilidad = calcular_u_a(aceleracion = aceleracion, tiempo_maniobra = tiempo_maniobra, v_final = v_final, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0, 10]
      v_final_1 = es_factible.iloc[0, 11]

      tiempo_maniobra_2 = es_factible.iloc[1, 10]
      v_final_2 = es_factible.iloc[1, 11]

      utilidad_1 = calcular_u_a(aceleracion = decision_autonomo, tiempo_maniobra = tiempo_maniobra_1, v_final = v_final_1, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      utilidad_2 = calcular_u_a(aceleracion = decision_autonomo, tiempo_maniobra = tiempo_maniobra_2, v_final = v_final_2, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)

      if utilidad_1 > utilidad_2:
        u = "utilidad 1"
      else:
        u = "utilidad 2"

      utilidad = max(utilidad_1, utilidad_2)
      print("Aceleración autónomo: {}, Aceleración tripulado: {}, Se ha elegido la utilidad {}".format(decision_autonomo, decision_humano, u))
      return(utilidad)

    else:
      return("Ha ocurrido un error")

### Calculemos la utilidad del vehículo con conductor

In [458]:
def calcular_utilidad_humano(decision_autonomo, decision_humano, x_a, v_a, x_h, v_h, x_final, v_min, v_max, b_h, w4, w5, w6, agresivo = False, d_seguridad = 5, t_maniobra_min = 5, inf_t = 0.01):
  """
  decision_autonomo se corresponde con la decisión que toma el ADS
  decisión humano se corresponde con la decisión del VH

  x_a: posición inicial vehículo autónomo
  v_a: velocidad inicial vehículo autónomo
  x_h: posición inicial vehículo con conductor
  v_h: velocidad inicial vehículo con conductor
  x_final: posición final de la vía de incorporación
  v_min: velocidad mínima de la vía
  v_max: velocidad máxima de la vía
  agresivo: Booleano que nos indica si el conductor del vehículo con conductor tiene una conducción agresiva
  d_seguridad: Distancia que tendrá en cuenta la función para considerar si la maniobra es segura
  t_maniobra_min: tiempo que se tarda en realizar la maniobra
  inf_t: infinitesimal de orden t a considerar

  b_h es el intercepto
  w4 es el peso correspondiente a la aceleración del vehículo humano
  w5 es el peso correspondiente a si se realiza o no la maniobra
  w6 es el peso correspondiente a si hay tiempo extra para la realización de la maniobra
  """
  # Es decir, si calculamos suponiendo que el autónomo toma la decisión de a_a = 0.3 ya suponemos que se cambia de carril, y por lo tanto que sí se realiza la maniobra. (ahhh no porque podría no realizarse la maniobra)

  if decision_autonomo == "No realizar la maniobra":
    utilidad = b_h + w4 * abs(decision_humano)

  ## Ahora tenemos que simular, porque aunque tengamos en cuenta que el vehículo autónomo cambia de carril, puede que la maniobra no se pueda efectuar y, por lo tanto,
  ## el valor final de la utilidad sería diferente. (al fin y al cabo esto va a ayudar a encontrar la mejor decisión para el vehículo con conductor)
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = maniobra_factible(x_a = x_a, v_a = v_a, a_a = decision_autonomo, x_h = x_h, v_h = v_h, a_h = decision_humano, x_final = x_final, v_min = v_min, v_max = v_max, agresivo = agresivo, d_seguridad = d_seguridad, t_maniobra_min = t_maniobra_min, inf_t = inf_t)

    # Ahora comprobemos cuál de los 3 resultados tenemos
    if es_factible.iloc[0, 0] == 0: # Es decir, que no se puede realizar la maniobra
      utilidad = b_h + w4 * abs(decision_humano)
      return utilidad

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      tiempo_maniobra = es_factible.iloc[0, 10]

      # Comprobemos que el tiempo es mayor que 8 para ver si aporta a la utilidad
      if tiempo_maniobra >= 8:
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0, 10]
      tiempo_maniobra_2 = es_factible.iloc[1, 10]

      if (tiempo_maniobra_1 >= 8) or (tiempo_maniobra_2 >= 8):
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra

  return utilidad

## Ejemplo: cálculo de utilidades, matrices del juego y equilibrios de Nash

Ahora vamos a probar cómo funcionan nuestras funciones de utilidad dando valores a los pesos y vemos que resultados obtenemos.

Para ello creemos la matriz de ganancias del ADS y creemos la matriz de ganancias del VH



### Ejemplo 1 Simultáneo: Carril de aceleración de 200 metros. Agresividad = True

In [459]:
pip install nashpy

In [460]:
import nashpy as nash

In [461]:
## Sea nuestro problema de valor inicial:
x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

## Tengamos nuestras matrices donde añadiremos las utilidades:

U_ADS = np.zeros((6, 5))

U_MV = np.zeros((6, 5))

## En primer lugar necesitamos definir los pesos de nuestras funciones.

# Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

# Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3


# Estos no los introducimos porque son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01

decisiones_ADS = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

# Ahora creamos un bucle que nos itere en las listas de decisiones para sacar las utilidades para cada decisión y para cada jugador
for index_a in range(len(decisiones_ADS)):
  decision_a = decisiones_ADS[index_a]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    U_ADS[index_a, index_h] = calcular_utilidad_autonomo(decision_autonomo = decision_a,
                                                         decision_humano = decision_h,
                                                         x_a = x_a0, v_a = v_a0,
                                                         x_h = x_h0, v_h = v_h0,
                                                         v_min = v_min, v_max = v_max,
                                                         x_final = x_fin_carril, agresivo = agresividad,
                                                         b_a = b_a, w1 = w1, w2 = w2, w3 = w3)
    U_MV[index_a, index_h] = calcular_utilidad_humano(decision_autonomo = decision_a,
                                                      decision_humano = decision_h,
                                                      x_a = x_a0, v_a = v_a0,
                                                      x_h = x_h0, v_h = v_h0,
                                                      v_min = v_min, v_max = v_max,
                                                      x_final = x_fin_carril, agresivo = agresividad,
                                                      b_h = b_h, w4 = w4, w5 = w5, w6 = w6)


U_MV = U_MV.round(decimals=3)

# Con esto obtenemos nuestras matrices de utilidades.

## Obtengamos ahora nuestro equilibrio de Nash:

game = nash.Game(U_ADS, U_MV)
print(game)


eqs = game.support_enumeration()
eqs

Bi matrix game with payoff matrices:

Row player:
[[0.   0.4  0.4  0.4  0.4 ]
 [0.5  0.5  0.5  0.5  0.5 ]
 [1.   1.   0.   0.7  0.7 ]
 [0.6  0.6  0.6  0.   0.  ]
 [0.5  0.5  0.5  0.   0.  ]
 [0.01 0.01 0.01 0.01 0.01]]

Column player:
[[0.  0.8 1.  0.8 0.6]
 [0.6 0.8 1.  0.8 0.6]
 [0.6 0.8 0.4 0.5 0.3]
 [0.3 0.5 0.7 0.2 0. ]
 [0.3 0.5 0.7 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]]


<generator object support_enumeration at 0x7d8415baad50>

In [462]:
## Veamos ahora los equilibrios de Nash

next(eqs)

(array([0., 0., 1., 0., 0., 0.]), array([0., 1., 0., 0., 0.]))

In [463]:
## Mostremos los resultados que hemos obtenido en el ejemplo 1:

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

a_a0 = 0
a_h0 = -1.5

print(maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max, agresivo = agresividad))


a_a0 = 1.5
a_h0 = 0

print(maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max, agresivo = agresividad))


   Maniobra  t_inicio_maniobra   x_a   v_a  a_a  x_h   v_h  a_h  dist_coches  \
0       1.0                0.0  15.0  20.0  0.0  0.0  24.0 -1.5         15.0   

   bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
0   1.0                     9.27                          20.0  
   Maniobra  t_inicio_maniobra   x_a   v_a  a_a  x_h   v_h  a_h  dist_coches  \
0       1.0                0.0  15.0  20.0  1.5  0.0  24.0  0.0         15.0   

   bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
0   1.0                     7.75                          25.0  


### Ejemplo 2 Simultáneo: Carril de aceleración de 120 metros. Agresividad True

In [464]:
## EJEMPLO 2

U_ADS = np.zeros((6, 5))

U_MV = np.zeros((6, 5))

# Ahora iteremos sobre nuestras listas de decisiones para poder añadir a las matrices las utilidades
# En primer lugar necesitamos definir los pesos de nuestras funciones.

## Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

## Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3

## Ahora introduzcamos nuestros valores iniciales del problema:
x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01

decisiones_ADS = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

# Ahora creamos un bucle que nos itere en las listas de decisiones para sacar las utilidades para cada decisión y para cada jugador
for index_a in range(len(decisiones_ADS)):
  decision_a = decisiones_ADS[index_a]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    U_ADS[index_a, index_h] = calcular_utilidad_autonomo(decision_autonomo = decision_a,
                                                         decision_humano = decision_h,
                                                         x_a = x_a0, v_a = v_a0,
                                                         x_h = x_h0, v_h = v_h0,
                                                         v_min = v_min, v_max = v_max,
                                                         x_final = x_fin_carril, agresivo = agresividad,
                                                         b_a = b_a, w1 = w1, w2 = w2, w3 = w3)
    U_MV[index_a, index_h] = calcular_utilidad_humano(decision_autonomo = decision_a,
                                                      decision_humano = decision_h,
                                                      x_a = x_a0, v_a = v_a0,
                                                      x_h = x_h0, v_h = v_h0,
                                                      v_min = v_min, v_max = v_max,
                                                      x_final = x_fin_carril, agresivo = agresividad,
                                                      b_h = b_h, w4 = w4, w5 = w5, w6 = w6)


U_MV = U_MV.round(decimals=3)


# Con esto obtenemos nuestras matrices de utilidades.

## Obtengamos ahora nuestro equilibrio de Nash:

game = nash.Game(U_ADS, U_MV)

print(game)

eqs = game.support_enumeration()
eqs

Bi matrix game with payoff matrices:

Row player:
[[0.   0.   0.1  0.1  0.1 ]
 [0.2  0.   0.   0.   0.  ]
 [0.7  0.7  0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.01 0.01 0.01 0.01 0.01]]

Column player:
[[0.  0.2 0.7 0.5 0.3]
 [0.3 0.2 0.4 0.2 0. ]
 [0.3 0.5 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]]


<generator object support_enumeration at 0x7d8415ba9f50>

In [465]:
## Veamos los equilibrios de Nash
next(eqs)

(array([1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0.]))

In [466]:
## Mostremos los resultados que hemos obtenido en el ejemplo 2:

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True

a_a0 = 0
a_h0 = -1.5

print(maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max, agresivo = agresividad))


a_a0 = -3
a_h0 = 0

print(maniobra_factible(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_min = v_min, v_max = v_max, agresivo = agresividad))


   Maniobra  t_inicio_maniobra   x_a   v_a  a_a  x_h   v_h  a_h  dist_coches  \
0       1.0                0.0  15.0  20.0  0.0  0.0  24.0 -1.5         15.0   

   bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
0   1.0                     5.26                          20.0  
   Maniobra  t_inicio_maniobra     x_a   v_a  a_a    x_h   v_h  a_h  \
0       1.0               2.56  56.375  12.5  0.0  61.44  24.0  0.0   

   dist_coches  bool  Tiempo para la maniobra  Velocidad ADS final maniobra  
0        5.065   1.0                     5.11                          12.5  


# 4. Modelo 2: Secuencial

En esta parte, haremos funciones que nos describirán la trayectoria del coche en función del tiempo. Y además tendremos funciones que nos proporcionen una lista con los booleanos de la distancia.

In [467]:
def trayectoria_a_trozos(x_a, v_a, a_a, x_h, v_h, a_h, x_final, t_ini, t_max, v_min, v_max, agresivo = False, inf_t = 0.01):
  """
  Esta función calculará las trayectorias de los dos vehículos y las devolverá en una lista

  x_a: posición inicial vehículo autónomo
  v_a: velocidad inicial vehículo autónomo
  a_a: aceleración inicial vehículo autónomo

  x_h: posición inicial vehículo con conductor
  v_h: velocidad inicial vehículo con conductor
  a_h: aceleración inicial vehículo con conductor

  x_final: posición final de la vía de incorporación

  t_ini: tiempo inicial
  t_max: tiempo final

  v_min: velocidad mínima de la vía
  v_max: velocidad máxima de la vía

  agresivo: Booleano que nos indica si el conductor del vehículo con conductor tiene una conducción agresiva
  inf_t: infinitesimal de orden t a considerar
  """

  if agresivo:
    v_max_h = v_max * 1.2 # Si especificamos que el conductor es agresivo, nos encontraremos que puede rebasar la velocidad máxima en un 20%
  else:
    v_max_h = v_max

  ## Ahora haremos nuestras predicciones segregando por tiempos. (siempre y cuando esté el coche autónomo dentro del carril de incorporación)

  resultados = [(t_ini, x_a, v_a, a_a, x_h, v_h, a_h)]

  while t_ini < t_max:

    # Comprobamos que estamos dentro del carril
    if x_final - x_a > 0:

      # Ahora comprobamos velocidades del vehículo autónomo
      if v_a >= v_max:
        v_a = v_max
        a_a = 0
      elif (v_a <= v_min) and (a_a < 0):
        v_a = v_min
        a_a = 0

      # Calculamos posiciones del coche autónomo
      x_a1, v_a1, a_a1 = posicion_coche(x_a, v_a, a_a, inf_t)

      # Comprobamos velocidades del coche no autónomo
      if v_h >= v_max_h:
        v_h = v_max_h
        a_h = 0
      elif (v_h <= v_min) and (a_h < 0): # Tenemos en cuenta también que el conductor nunca va a bajar la velocidad mínima de la vía
        v_h = v_min
        a_h = 0

      # Calculamos posiciones del coche humano
      x_h1, v_h1, a_h1 = posicion_coche(x_h, v_h, a_h, inf_t)


      t_ini += inf_t
      resultados.append((t_ini, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1))

      x_a, v_a, a_a = x_a1, v_a1, a_a1
      x_h, v_h, a_h = x_h1, v_h1, a_h1

    else:
      t_ini = t_max


  resultados_np = np.array(resultados)

  # Ahora calculamos la distancia entre las posiciones de los coches
  diferencia = abs(resultados_np[:,1] - resultados_np[:,4]).reshape(-1, 1)

  # Añadimos la columna a nuestra matriz de resultados
  resultados_np = np.hstack((resultados_np, diferencia))


  return resultados_np



In [468]:
## Probemos a ver si funciona:

t_ini = 0
t_max = 1.99

x_a0 = 0
v_a0 = 18
x_h0 = 10
v_h0 = 25

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

a_a0 = 1.5
a_h0 = 0


trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                     x_h = x_h0, v_h = v_h0, a_h = a_h0,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)

trayectoria_0_2

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

t_ini = 2
t_max = 20

a_a1 = 1.5
a_h1 = 0

trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                     x_h = x_h1, v_h = v_h1, a_h = a_h1,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)



trayectoria_2_fin

trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))



In [469]:
## Calculemos ahora si se puede realizar la maniobra en función de las decisiones y cómo afectan

def maniobra_factible_secuencial(trayectoria, t_maniobra_min = 5, d_seguridad = 5, inf_t = 0.01):

  trayectoria_np = np.array(trayectoria)
  diferencia = trayectoria_np[:, -1]

# Ahora veamos qué distancias son superiores a la distancia de seguridad (5 (metros))
  superiores = (diferencia > d_seguridad).astype(int).reshape(-1, 1)
  trayectoria_np = np.hstack((trayectoria_np, superiores))

  ## Ahora tenemos que ver que esa condición se cumpla durante el t_maniobra_min
  # Para ello tenemos que tener en cuenta de cuánto es nuestro infinitesimal de orden t para
  # calcular el número de True's que tienen que ir seguidos en la columna superiores

  # Calculemos cuantos infinitesimales con True tienen que ir seguidos en funcion de nuestro t_maniobra_min
  num_seguidos = int(t_maniobra_min / inf_t)

  ## Usemos la función creada anteriormente para contar los valores consecutivos de nuestra lista superiores
  secuencias, indices_inicio_secuencia = contar_consecutivos(superiores.flatten(), num_seguidos)


  if secuencias != []:
    # Obtengamos las posiciones y los tiempos en los que el vehículo autónomo puede comenzar la maniobra, así como los
    # segundos de los que dispondrá para realizar dicha maniobra
    resultados = [['Maniobra', "t_inicio_maniobra", "x_a", "v_a", "a_a", "x_h", "v_h", "a_h", "dist_coches", "bool", "Tiempo para la maniobra", "Velocidad ADS final maniobra"]]

    for indice in range(len(secuencias)):
      tiempo_disp_maniobra = secuencias[indice]*inf_t
      i_ini_maniobra = indices_inicio_secuencia[indice]
      velocidad_a_final_maniobra = trayectoria_np[i_ini_maniobra + int(num_seguidos)].tolist()[2] # Sumamos num_seguidos para saber la velocidad final de la maniobra
      valores_vehiculos = trayectoria_np[i_ini_maniobra].tolist()
      resultado_maniobra = [True] + valores_vehiculos + [tiempo_disp_maniobra] + [velocidad_a_final_maniobra]
      resultados.append(resultado_maniobra)

    #print("Se puede realizar la maniobra")
    return pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0]))
    ### return("Se puede realizar la maniobra", pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))
    #return(pd.DataFrame(lista_valores_np))

  else:
    resultados = [['Maniobra']]
    ### return("No se puede realizar la maniobra")
    resultados.append([False])
    return(pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))

In [470]:

## probemos si funciona

maniobra_factible_secuencial(trayectoria_0_fin)

,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,0.0,0.0,18.0,1.5,10.0,25.0,0.0,10.0,1.0,8.68,25.0


## Funciones de utilidad para el modelo secuencial

Las funciones de utilidad son las mismas que en el modelo simultáneo, sin embargo, es necesario modificarlas para adaptarlas a este procedimiento.


In [471]:
def calcular_utilidad_autonomo_secuencial(resultado, decision_autonomo, b_a, w1, w2, w3):
  """
  resultado
  decision_autonomo es la decisión que toma el ADS
  b_a es el intercepto
  w1 es el peso correspondiente a la aceleración
  w2 es el peso correspondiente al tiempo extra
  w3 es el peso correspondiente a la velocidad alcanzada tras la maniobra
  """

  # En primer lugar veamos qué ocurre si la decision es no realizar la maniobra.
  if decision_autonomo == 'No realizar la maniobra':
    return 0.01

  # Ahora veamos qué ocurre cuando la decisión es realizarla
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = resultado
    if es_factible.iloc[0,0] == 0: # Es decir, que no se puede realizar la maniobra
      return 0

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      aceleracion = decision_autonomo
      tiempo_maniobra = es_factible.iloc[0,10]
      v_final = es_factible.iloc[0,11]

      utilidad = calcular_u_a(aceleracion = aceleracion, tiempo_maniobra = tiempo_maniobra, v_final = v_final, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0,10]
      v_final_1 = es_factible.iloc[0,11]

      tiempo_maniobra_2 = es_factible.iloc[1,10]
      v_final_2 = es_factible.iloc[1,11]

      utilidad_1 = calcular_u_a(aceleracion = decision_autonomo, tiempo_maniobra = tiempo_maniobra_1, v_final = v_final_1, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      utilidad_2 = calcular_u_a(aceleracion = decision_autonomo, tiempo_maniobra = tiempo_maniobra_2, v_final = v_final_2, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)

      if utilidad_1 > utilidad_2:
        u = "utilidad 1"
      else:
        u = "utilidad 2"

      utilidad = max(utilidad_1, utilidad_2)
      print("Aceleración autónomo: {}, Aceleración tripulado: {}, Se ha elegido la utilidad {}".format(decision_autonomo, decision_humano, u))
      return(utilidad)

    else:
      return("Ha ocurrido un error")

In [472]:
def calcular_utilidad_humano_secuencial(resultado, decision_autonomo, decision_humano, b_h, w4, w5, w6):
  """
  resultado
  decision_humano es la aceleración que puede tomar el humano
  b_h es el intercepto
  w4 es el peso correspondiente a la aceleración del vehículo humano
  w5 es el peso correspondiente a si se realiza o no la maniobra
  w6 es el peso correspondiente a si hay tiempo extra para la realización de la maniobra
  """
  # Es decir, si calculamos suponiendo que el autónomo toma la decisión de a_a = 0.3 ya suponemos que se cambia de carril, y por lo tanto que sí se realiza la maniobra. (ahhh no porque podría no realizarse la maniobra)

  if decision_autonomo == "No realizar la maniobra":
    utilidad = b_h + w4 * abs(decision_humano)

  ## Ahora tenemos que simular, porque aunque tengamos en cuenta que el vehículo autónomo cambia de carril, puede que la maniobra no se pueda efectuar y, por lo tanto,
  ## el valor final de la utilidad sería diferente. (al fin y al cabo esto va a ayudar a encontrar la mejor decisión para el vehículo con conductor)
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = resultado

    # Ahora comprobemos cuál de los 3 resultados tenemos
    if es_factible.iloc[0,0] == 0: # Es decir, que no se puede realizar la maniobra
      utilidad = b_h + w4 * abs(decision_humano)
      return utilidad

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      tiempo_maniobra = es_factible.iloc[0,10]

      # Comprobemos que el tiempo es mayor que 8 para ver si aporta a la utilidad
      if tiempo_maniobra >= 8:
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0,10]
      tiempo_maniobra_2 = es_factible.iloc[1,10]

      if (tiempo_maniobra_1 >= 8) or (tiempo_maniobra_2 >= 8):
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra

  return utilidad

### Ejemplo 1 Secuencial: Carril de aceleración de 200 metros, Agresividad = True

In [473]:
## Para ello creemos dos matriz de 0's donde iremos guardando la información

U_ADS = np.zeros((6, 5))

U_MV = np.zeros((6, 5))

# Ahora iteremos sobre nuestras listas de decisiones para poder añadir a las matrices las utilidades
# En primer lugar necesitamos definir los pesos de nuestras funciones.

## Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

## Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3

## Ahora introduzcamos nuestros valores iniciales del problema:
x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01

decisiones_ADS = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

# Ahora creamos un bucle que nos itere en las listas de decisiones para sacar las utilidades para cada decisión y para cada jugador
for index_a in range(len(decisiones_ADS)):
  decision_a = decisiones_ADS[index_a]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    if decision_a == 'No realizar la maniobra':
      U_ADS[index_a, index_h] = 0.01
      U_MV[index_a, index_h] = b_h + w4 * abs(decision_h)

    else:
      t_ini = 0
      t_max = 1.99

      a_a0 = decision_a
      a_h0 = 0

      trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                          x_h = x_h0, v_h = v_h0, a_h = a_h0,
                          x_final = x_fin_carril,
                          t_ini = t_ini, t_max = t_max,
                          v_min = v_min, v_max = v_max,
                          agresivo = agresividad)

      (t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

      t_ini = 2
      t_max = 20

      a_a1 = decision_a
      a_h1 = decision_h

      trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                          x_h = x_h1, v_h = v_h1, a_h = a_h1,
                          x_final = x_fin_carril,
                          t_ini = t_ini, t_max = t_max,
                          v_min = v_min, v_max = v_max,
                          agresivo = agresividad)

      trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))
      trayectoria = maniobra_factible_secuencial(trayectoria_0_fin)

      U_ADS[index_a, index_h] = calcular_utilidad_autonomo_secuencial(resultado = trayectoria,
                                                                      decision_autonomo = decision_a,
                                                                      b_a = b_a, w1 = w1, w2 = w2, w3 = w3)

      U_MV[index_a, index_h] = calcular_utilidad_humano_secuencial(resultado = trayectoria,
                                                        decision_autonomo = decision_a,
                                                        decision_humano = decision_h,
                                                        b_h = b_h, w4 = w4, w5 = w5, w6 = w6)


U_MV = U_MV.round(decimals=3)
print(U_ADS)
print(U_MV)

# Con esto obtenemos nuestras matrices de utilidades.

## Obtengamos ahora nuestro equilibrio de Nash:

game = nash.Game(U_ADS, U_MV)

print(game)

eqs = game.support_enumeration()
eqs

[[0.4  0.4  0.4  0.4  0.4 ]
 [0.5  0.5  0.5  0.5  0.5 ]
 [0.7  0.   0.   0.7  0.7 ]
 [0.6  0.6  0.6  0.   0.  ]
 [0.5  0.5  0.5  0.5  0.  ]
 [0.01 0.01 0.01 0.01 0.01]]
[[0.6 0.8 1.  0.8 0.6]
 [0.6 0.8 1.  0.8 0.6]
 [0.3 0.2 0.4 0.5 0.3]
 [0.3 0.5 0.7 0.2 0. ]
 [0.3 0.5 0.7 0.5 0. ]
 [0.  0.2 0.4 0.2 0. ]]
Bi matrix game with payoff matrices:

Row player:
[[0.4  0.4  0.4  0.4  0.4 ]
 [0.5  0.5  0.5  0.5  0.5 ]
 [0.7  0.   0.   0.7  0.7 ]
 [0.6  0.6  0.6  0.   0.  ]
 [0.5  0.5  0.5  0.5  0.  ]
 [0.01 0.01 0.01 0.01 0.01]]

Column player:
[[0.6 0.8 1.  0.8 0.6]
 [0.6 0.8 1.  0.8 0.6]
 [0.3 0.2 0.4 0.5 0.3]
 [0.3 0.5 0.7 0.2 0. ]
 [0.3 0.5 0.7 0.5 0. ]
 [0.  0.2 0.4 0.2 0. ]]


<generator object support_enumeration at 0x7d8415baaf10>

In [474]:
next(eqs)

(array([0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 1., 0.]))

In [475]:
## Mostremos los resultados que hemos obtenido en el ejemplo 1:

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

t_ini = 0
t_max = 1.99

a_a0 = 0
a_h0 = 0

trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                    x_h = x_h0, v_h = v_h0, a_h = a_h0,
                    x_final = x_fin_carril,
                    t_ini = t_ini, t_max = t_max,
                    v_min = v_min, v_max = v_max,
                    agresivo = agresividad)

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

t_ini = 2
t_max = 20

a_a1 = 0
a_h1 = 1.5

trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                    x_h = x_h1, v_h = v_h1, a_h = a_h1,
                    x_final = x_fin_carril,
                    t_ini = t_ini, t_max = t_max,
                    v_min = v_min, v_max = v_max,
                    agresivo = agresividad)

trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))
maniobra_factible_secuencial(trayectoria_0_fin)



,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,4.15,97.8,20.0,0.0,102.826875,27.225,1.5,5.026875,1.0,5.13,20.0


### Ejemplo 2 Secuencial: Carril de aceleración de 120 metros, Agresividad = True

In [476]:
## Para ello creemos dos matriz de 0's donde iremos guardando la información

U_ADS = np.zeros((6, 5))

U_MV = np.zeros((6, 5))

# Ahora iteremos sobre nuestras listas de decisiones para poder añadir a las matrices las utilidades
# En primer lugar necesitamos definir los pesos de nuestras funciones.

## Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

## Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3

## Ahora introduzcamos nuestros valores iniciales del problema:
x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True

# Estos no los introducimos pq son los valores que tenemos por defecto en nuestra función
#d_seguridad = 5
#t_maniobra_min = 5
#inf_t = 0.01

decisiones_ADS = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

# Ahora creamos un bucle que nos itere en las listas de decisiones para sacar las utilidades para cada decisión y para cada jugador
for index_a in range(len(decisiones_ADS)):
  decision_a = decisiones_ADS[index_a]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    if decision_a == 'No realizar la maniobra':
      U_ADS[index_a, index_h] = 0.01
      U_MV[index_a, index_h] = b_h + w4 * abs(decision_h)

    else:
      t_ini = 0
      t_max = 1.99

      a_a0 = decision_a
      a_h0 = 0

      trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                          x_h = x_h0, v_h = v_h0, a_h = a_h0,
                          x_final = x_fin_carril,
                          t_ini = t_ini, t_max = t_max,
                          v_min = v_min, v_max = v_max,
                          agresivo = agresividad)

      (t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

      t_ini = 2
      t_max = 20

      a_a1 = decision_a
      a_h1 = decision_h

      trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                          x_h = x_h1, v_h = v_h1, a_h = a_h1,
                          x_final = x_fin_carril,
                          t_ini = t_ini, t_max = t_max,
                          v_min = v_min, v_max = v_max,
                          agresivo = agresividad)

      trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))
      trayectoria = maniobra_factible_secuencial(trayectoria_0_fin)

      U_ADS[index_a, index_h] = calcular_utilidad_autonomo_secuencial(resultado = trayectoria,
                                                                      decision_autonomo = decision_a,
                                                                      b_a = b_a, w1 = w1, w2 = w2, w3 = w3)

      U_MV[index_a, index_h] = calcular_utilidad_humano_secuencial(resultado = trayectoria,
                                                        decision_autonomo = decision_a,
                                                        decision_humano = decision_h,
                                                        b_h = b_h, w4 = w4, w5 = w5, w6 = w6)


U_MV = U_MV.round(decimals=3)


# Con esto obtenemos nuestras matrices de utilidades.

## Obtengamos ahora nuestro equilibrio de Nash:

game = nash.Game(U_ADS, U_MV)

print(game)

eqs = game.support_enumeration()
eqs

Bi matrix game with payoff matrices:

Row player:
[[0.1  0.1  0.1  0.1  0.1 ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.  ]
 [0.01 0.01 0.01 0.01 0.01]]

Column player:
[[0.3 0.5 0.7 0.5 0.3]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]
 [0.  0.2 0.4 0.2 0. ]]


<generator object support_enumeration at 0x7d84174f6110>

In [477]:
next(eqs)

(array([1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0.]))

In [478]:
## Mostremos los resultados que hemos obtenido en el ejemplo 2:

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True


t_ini = 0
t_max = 1.99

a_a0 = -3
a_h0 = 0

trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                    x_h = x_h0, v_h = v_h0, a_h = a_h0,
                    x_final = x_fin_carril,
                    t_ini = t_ini, t_max = t_max,
                    v_min = v_min, v_max = v_max,
                    agresivo = agresividad)

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

t_ini = 2
t_max = 20

a_a1 = -3
a_h1 = 0

trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                    x_h = x_h1, v_h = v_h1, a_h = a_h1,
                    x_final = x_fin_carril,
                    t_ini = t_ini, t_max = t_max,
                    v_min = v_min, v_max = v_max,
                    agresivo = agresividad)

trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))
maniobra_factible_secuencial(trayectoria_0_fin)


,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,2.57,56.375,12.5,0.0,61.44,24.0,0.0,5.065,1.0,5.11,12.5


# 6. Modelo 3: Mixto

In [479]:
def calcular_u_a_mixto(aceleracion_1, aceleracion_2, tiempo_maniobra, v_final, v_max, b_a, w1, w2, w3):
  """
  aceleración es la aceleración del vehículo autónomo
  tiempo_maniobra es el tiempo total que existe para realizar la maniobra
  v_final es la velocidad que alcanza el coche autónomo una vez la maniobra ha finalizado

  b_a es el intercepto
  w1 es el peso correspondiente a la aceleración
  w2 es el peso correspondiente al tiempo extra
  w3 es el peso correspondiente a la velocidad alcanzada tras la maniobra
  """
  # En primer lugar calculamos la indicatriz de la maniobra
  if tiempo_maniobra >= 8:
    I_t_extra = 1
  else:
    I_t_extra = 0

  # En segundo lugar calculamos la indicatriz de la velocidad final
  if v_max * 0.8 <= v_final and v_final <= v_max:
    I_vel = 1
  else:
    I_vel = 0

  # Finalmente calculamos la utilidad y la devolvemos
  utilidad = b_a + w1 * (abs(aceleracion_1) + abs(aceleracion_2)) * (1/2) + w2 * I_t_extra + w3 * I_vel
  return utilidad

In [480]:
# En primer lugar debemos definir una función que nos calcule la utilidad mixta con las dos decisiones

def calcular_utilidad_autonomo_mixto(resultado, decision_autonomo_1, decision_autonomo_2, b_a, w1, w2, w3):
  """
  resultado
  decision_autonomo_1 es la decisión inicial que toma el ADS
  decision_autonomo_2 es la decisión que toma el ADS a los 3 segundos
  b_a es el intercepto
  w1 es el peso correspondiente a la aceleración
  w2 es el peso correspondiente al tiempo extra
  w3 es el peso correspondiente a la velocidad alcanzada tras la maniobra
  """

  # En primer lugar veamos qué ocurre si la decision es no realizar la maniobra.
  if decision_autonomo_1 == 'No realizar la maniobra' or decision_autonomo_2 == 'No realizar la maniobra':
    return 0.01

  # Ahora veamos qué ocurre cuando la decisión es realizarla
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = resultado
    if es_factible.iloc[0,0] == 0: # Es decir, que no se puede realizar la maniobra
      return 0

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      aceleracion_1 = decision_autonomo_1
      aceleracion_2 = decision_autonomo_2
      tiempo_maniobra = es_factible.iloc[0,10]
      v_final = es_factible.iloc[0,11]

      utilidad = calcular_u_a_mixto(aceleracion_1 = aceleracion_1, aceleracion_2 = aceleracion_2, tiempo_maniobra = tiempo_maniobra, v_final = v_final, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0,10]
      v_final_1 = es_factible.iloc[0,11]

      tiempo_maniobra_2 = es_factible.iloc[1,10]
      v_final_2 = es_factible.iloc[1,11]

      utilidad_1 = calcular_u_a_mixto(aceleracion_1 = decision_autonomo_1, aceleracion_2 = decision_autonomo_2, tiempo_maniobra = tiempo_maniobra_1, v_final = v_final_1, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)
      utilidad_2 = calcular_u_a_mixto(aceleracion_1 = decision_autonomo_1, aceleracion_2 = decision_autonomo_2, tiempo_maniobra = tiempo_maniobra_2, v_final = v_final_2, v_max = v_max, w1 = w1, w2 = w2, w3 = w3, b_a = b_a)

      if utilidad_1 > utilidad_2:
        u = "utilidad 1"
      else:
        u = "utilidad 2"

      utilidad = max(utilidad_1, utilidad_2)
      print("Aceleración autónomo: {}, Aceleración tripulado: {}, Se ha elegido la utilidad {}".format(decision_autonomo, decision_humano, u))
      return(utilidad)

    else:
      return("Ha ocurrido un error")


In [481]:
# Ahora definimos la utilidad del vehículo tripulado

def calcular_utilidad_humano_mixto(resultado, decision_autonomo_1, decision_autonomo_2, decision_humano, b_h, w4, w5, w6):
  """
  resultado
  decision_humano es la aceleración que puede tomar el humano
  decision_autonomo_1 es la primera decisión del ADS
  decision_autonomo_2 es la segunda decision del ADS
  b_h es el intercepto
  w4 es el peso correspondiente a la aceleración del vehículo humano
  w5 es el peso correspondiente a si se realiza o no la maniobra
  w6 es el peso correspondiente a si hay tiempo extra para la realización de la maniobra
  """
  # Es decir, si calculamos suponiendo que el autónomo toma la decisión de a_a = 0.3 ya suponemos que se cambia de carril, y por lo tanto que sí se realiza la maniobra. (ahhh no porque podría no realizarse la maniobra)

  if decision_autonomo_1 == "No realizar la maniobra" or decision_autonomo_2 == 'No realizar la maniobra':
    utilidad = b_h + w4 * abs(decision_humano)

  ## Ahora tenemos que simular, porque aunque tengamos en cuenta que el vehículo autónomo cambia de carril, puede que la maniobra no se pueda efectuar y, por lo tanto,
  ## el valor final de la utilidad sería diferente. (al fin y al cabo esto va a ayudar a encontrar la mejor decisión para el vehículo con conductor)
  else:
    ## En primer lugar calculamos la simulación y vemos qué resultados nos da (los posibles son: Maniobra_factible = False, Maniobra_factible = 1 solución, Maniobra_factible = dos soluciones)
    es_factible = resultado

    # Ahora comprobemos cuál de los 3 resultados tenemos
    if es_factible.iloc[0,0] == 0: # Es decir, que no se puede realizar la maniobra
      utilidad = b_h + w4 * abs(decision_humano)
      return utilidad

    elif es_factible.shape[0] == 1: # Es decir, descartamos que sea 0 (en el paso anterior) por lo tanto es 1, y además, con el shape 1, forzamos a que la salida sea solo 1 maniobra.
      tiempo_maniobra = es_factible.iloc[0,10]

      # Comprobemos que el tiempo es mayor que 8 para ver si aporta a la utilidad
      if tiempo_maniobra >= 8:
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra
      return utilidad

    elif es_factible.shape[0] == 2: # Ahora calculamos la mayor utilidad si hay dos posibles maniobras.
      tiempo_maniobra_1 = es_factible.iloc[0,10]
      tiempo_maniobra_2 = es_factible.iloc[1,10]

      if (tiempo_maniobra_1 >= 8) or (tiempo_maniobra_2 >= 8):
        I_t_extra = 1
      else:
        I_t_extra = 0

      utilidad = b_h + w4 * abs(decision_humano) + w5 + w6 * I_t_extra

  return utilidad

## Trayectoria Mixto


In [482]:
## Para poder ver las trayectorias, usemos la misma función que teníamos en el secuencial:

# Si deseamos calcular la trayectoria en los 3 primeros segundos, y luego variar la aceleración del ADS,
# hagámoslo por separado y luego juntemos los dos vectores.

# Ejemplo con los valores iniciales en los 3 primeros segundos
t_ini = 0
t_max = 2.99

x_a0 = 0
v_a0 = 18
x_h0 = 10
v_h0 = 25

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

a_a0 = 1.5
a_h0 = 3


trayectoria_0_3 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                     x_h = x_h0, v_h = v_h0, a_h = a_h0,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)

trayectoria_0_3

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_3[-1]

t_ini = 3
t_max = 20

a_a1 = -3
a_h1 = 3

trayectoria_3_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                     x_h = x_h1, v_h = v_h1, a_h = a_h1,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)



trayectoria_3_fin

trayectoria_0_fin = np.vstack((trayectoria_0_3, trayectoria_3_fin))

# Con esto vemos que nuestra función funciona para describir el comportamiento deseado.

# Además, usemos también la función maniobra_factible_secuencial, que es la que comprueba si la maniobra es factible


### Ejemplo 1 Mixto: Carril de aceleración de 200 metros, Agresividad = True

In [483]:
# En este modelo lo que va a ocurrir es que los vehículos van a tomar en primer
# lugar una decisión simultáneamente, y, tres segundos después, el ADS tomará otra decisión

# Para ello definamos en primer lugar A1 y A2, que serán las decisiones que podrá tomar el ADS (las mismas en ambos casos)

decisiones_ADS_1 = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']
decisiones_ADS_2 = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

# Las separamos para entender mejor posteriormente los bucles

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

## Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

## Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3

lista_utilidades = []

# Creemos una lista donde guardar las utilidades, en el caso anterior, teníamos 30 utilidades, es este, tendremos 180

for index_a_1 in range(len(decisiones_ADS_1)):
  decision_a_1 = decisiones_ADS_1[index_a_1]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    for index_a_2 in range(len(decisiones_ADS_2)):
      decision_a_2 = decisiones_ADS_2[index_a_2]

      # Primero obtenemos todas las combinaciones posibles de decisiones que nos podemos encontrar
      if decision_a_1 == 'No realizar la maniobra' or decision_a_2 == 'No realizar la maniobra':
        lista_utilidades.append((decision_a_1, decision_h, decision_a_2, 0.01, b_h + w4 * abs(decision_h)))

      # En caso de que se realice la maniobra, tenemos que simular, y por lo tanto tendremos que:
      else:
        t_ini = 0
        t_max = 2.99

        x_a0 = 15
        v_a0 = 20
        x_h0 = 0
        v_h0 = 24

        v_min = 12.5
        v_max = 25
        x_fin_carril = 200
        agresividad = True

        a_a0 = decision_a_1
        a_h0 = decision_h

        trayectoria_0_3 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                            x_h = x_h0, v_h = v_h0, a_h = a_h0,
                            x_final = x_fin_carril,
                            t_ini = t_ini, t_max = t_max,
                            v_min = v_min, v_max = v_max,
                            agresivo = agresividad)

        (t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_3[-1]

        t_ini = 3
        t_max = 20

        a_a1 = decision_a_2
        a_h1 = a_h1

        trayectoria_3_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                            x_h = x_h1, v_h = v_h1, a_h = a_h1,
                            x_final = x_fin_carril,
                            t_ini = t_ini, t_max = t_max,
                            v_min = v_min, v_max = v_max,
                            agresivo = agresividad)

        trayectoria_0_fin = np.vstack((trayectoria_0_3, trayectoria_3_fin))
        trayectoria = maniobra_factible_secuencial(trayectoria_0_fin)

        ## Con esto, solamente nos queda calcular las utilidades:

        util_ADS = calcular_utilidad_autonomo_mixto(resultado = trayectoria,
                                                    decision_autonomo_1 = decision_a_1,
                                                    decision_autonomo_2 = decision_a_2,
                                                    b_a = b_a, w1 = w1, w2 = w2, w3 = w3)

        util_H = calcular_utilidad_humano_mixto(resultado = trayectoria,
                                                decision_autonomo_1 = decision_a_1,
                                                decision_autonomo_2 = decision_a_2,
                                                decision_humano = decision_h,
                                                b_h = b_h, w4 = w4, w5 = w5, w6 = w6)

        lista_utilidades.append((decision_a_1, decision_h, decision_a_2, util_ADS, util_H))


lista_utilidades

pd.DataFrame(lista_utilidades).to_excel("utilidades.xlsx", sheet_name="Carril 200 metros")



### Ejemplo 2: Carril de 120 metros, Agresividad = True

In [484]:
# En este modelo lo que va a ocurrir es que los vehículos van a tomar en primer
# lugar una decisión simultáneamente, y, tres segundos después, el ADS tomará otra decisión

# Para ello definamos en primer lugar A1 y A2, que serán las decisiones que podrá tomar el ADS (las mismas en ambos casos)

decisiones_ADS_1 = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']
decisiones_ADS_2 = [-3, -1.5, 0, 1.5, 3, 'No realizar la maniobra']

# Las separamos para entender mejor posteriormente los bucles

decisiones_VH = [-3, -1.5, 0, 1.5, 3]

## Pesos de la función de utilidad del vehículo autónomo
b_a, w1, w2, w3 = 0.3, -0.2/3, 0.3, 0.4

## Pesos de la función de utilidad del vehículo con conductor
b_h, w4, w5, w6 = 0.40, -0.40/3 , 0.3, 0.3

lista_utilidades = []

# Creemos una lista donde guardar las utilidades, en el caso anterior, teníamos 30 utilidades, es este, tendremos 180

for index_a_1 in range(len(decisiones_ADS_1)):
  decision_a_1 = decisiones_ADS_1[index_a_1]

  for index_h in range(len(decisiones_VH)):
    decision_h = decisiones_VH[index_h]

    for index_a_2 in range(len(decisiones_ADS_2)):
      decision_a_2 = decisiones_ADS_2[index_a_2]

      # Primero obtenemos todas las combinaciones posibles de decisiones que nos podemos encontrar
      if decision_a_1 == 'No realizar la maniobra' or decision_a_2 == 'No realizar la maniobra':
        lista_utilidades.append((decision_a_1, decision_h, decision_a_2, 0.01, b_h + w4 * abs(decision_h)))

      # En caso de que se realice la maniobra, tenemos que simular, y por lo tanto tendremos que:
      else:
        t_ini = 0
        t_max = 2.99

        x_a0 = 15
        v_a0 = 20
        x_h0 = 0
        v_h0 = 24

        v_min = 12.5
        v_max = 25
        x_fin_carril = 120
        agresividad = True

        a_a0 = decision_a_1
        a_h0 = decision_h

        trayectoria_0_3 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                            x_h = x_h0, v_h = v_h0, a_h = a_h0,
                            x_final = x_fin_carril,
                            t_ini = t_ini, t_max = t_max,
                            v_min = v_min, v_max = v_max,
                            agresivo = agresividad)

        (t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_3[-1]

        t_ini = 3
        t_max = 20

        a_a1 = decision_a_2
        a_h1 = a_h1

        trayectoria_3_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                            x_h = x_h1, v_h = v_h1, a_h = a_h1,
                            x_final = x_fin_carril,
                            t_ini = t_ini, t_max = t_max,
                            v_min = v_min, v_max = v_max,
                            agresivo = agresividad)

        trayectoria_0_fin = np.vstack((trayectoria_0_3, trayectoria_3_fin))
        trayectoria = maniobra_factible_secuencial(trayectoria_0_fin)

        ## Con esto, solamente nos queda calcular las utilidades:

        util_ADS = calcular_utilidad_autonomo_mixto(resultado = trayectoria,
                                                    decision_autonomo_1 = decision_a_1,
                                                    decision_autonomo_2 = decision_a_2,
                                                    b_a = b_a, w1 = w1, w2 = w2, w3 = w3)

        util_H = calcular_utilidad_humano_mixto(resultado = trayectoria,
                                                decision_autonomo_1 = decision_a_1,
                                                decision_autonomo_2 = decision_a_2,
                                                decision_humano = decision_h,
                                                b_h = b_h, w4 = w4, w5 = w5, w6 = w6)

        lista_utilidades.append((decision_a_1, decision_h, decision_a_2, util_ADS, util_H))


lista_utilidades

pd.DataFrame(lista_utilidades).to_excel("utilidades1.xlsx", sheet_name="Carril 120 metros")



# GIF's de los resultados

En esta sección encontramos una serie de funciones con las cuales obtenemos o gráficas con las trayectorias de los vehículos o un gif de las posiciones de los vehículos.

Para obtenerla de un ejemplo concreto, hay que introducir: los parámetros necesarios del problema (aceleraciones incluídas)y el t_0 en el que se inicia la maniobra.

In [485]:
## En primer lugar, adaptamos nuestra función que calcula si la maniobra es segura para que nos devuelva las posiciones de los vehículos

def calcular_posiciones(x_a, v_a, a_a, x_h, v_h, a_h, x_final, v_min, v_max, agresivo = False, d_seguridad = 5, t_maniobra_min = 5, inf_t = 0.01):
  """
  Esta función nos devuelve las posiciones de los vehículos

  x_a: posición inicial vehículo autónomo
  v_a: velocidad inicial vehículo autónomo
  a_a: aceleración inicial vehículo autónomo

  x_h: posición inicial vehículo con conductor
  v_h: velocidad inicial vehículo con conductor
  a_h: aceleración inicial vehículo con conductor

  x_final: posición final de la vía de incorporación

  v_min: velocidad mínima de la vía
  v_max: velocidad máxima de la vía

  agresivo: Booleano que nos indica si el conductor del vehículo con conductor tiene una conducción agresiva
  d_seguridad: Distancia que tendrá en cuenta la función para considerar si la maniobra es segura
  t_maniobra_min: tiempo que se tarda en realizar la maniobra
  inf_t: infinitesimal de orden t a considerar
  """

  # Creemos un bucle cuya condicicón sea que no se nos finalize el carril de aceleración
  # (para ello restemos la distancia del vehículo autónomo menos la distancia del punto final del carril)

  if agresivo:
    v_max_h = v_max * 1.2 # Si especificamos que el conductor es agresivo, nos encontraremos que puede rebasar la velocidad máxima en un 20%
  else:
    v_max_h = v_max

  t_ini = 0 # Definamos el tiempo inicial
  lista_valores = [[t_ini, x_a, v_a, a_a, x_h, v_h, a_h]] # Definamos nuestra lista de resultados y le añadimos los datos iniciales

  while x_final - x_a > 0:

    # Calculemos las posiciones de nuestro vehículo autónomo (tendremos en cuenta velocidad máxima y velocidad mínima)
    if v_a >= v_max:
      v_a = v_max
      a_a = 0
    elif (v_a <= v_min) and (a_a < 0):
      v_a = v_min
      a_a = 0

    x_a1, v_a1, a_a1 = posicion_coche(x_a, v_a, a_a, inf_t)

    # Calculemos las posiciones de nuestro vehículo humano
    if v_h >= v_max_h:
      v_h = v_max_h
      a_h = 0
    elif (v_h <= v_min) and (a_h < 0): # Tenemos en cuenta también que el conductor nunca va a bajar la velocidad mínima de la vía
      v_h = v_min
      a_h = 0

    x_h1, v_h1, a_h1 = posicion_coche(x_h, v_h, a_h, inf_t)

    # Sumemos el infinitesimal de orden t a nuestro tiempo inicial
    t_ini += inf_t

    # Añadamos los resultados a nuestra lista de valores y actualicemos estos para que pueda continuar el bucle
    lista_valores.append([t_ini, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1])

    x_a, v_a, a_a, x_h, v_h, a_h = x_a1, v_a1, a_a1, x_h1, v_h1, a_h1


  ## Obtengamos los resultados
  return lista_valores


In [486]:
## MODELO SIMULTÁNEO, Ejemplo 1.

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True

#Resultados Eq Nash:

a_a0 = -3
a_h0 = 0

lista_valores = calcular_posiciones(x_a = x_a0, v_a = v_a0, a_a = a_a0, x_h = x_h0, v_h = v_h0, a_h = a_h0, x_final = x_fin_carril, v_max = v_max, v_min = v_min)

lista_valores = np.array(lista_valores)
posicion_x_ADS = lista_valores[:,1]
posicion_x_ADS

posicion_x_VC = lista_valores[:,4]
#list(posicion_x_VC)

In [487]:
## Creamos una función que recorra una lista que nos añada a la posición y lo que necesitamos


# Maniobra cambio de carril: 5 segundos
# Pasar en ese tiempo de y=0 a y=1

def obtener_coordenadas_y(len_x, t_0):
    '''
    Esta función devuelve la lista de valores del eje y para realizar la maniobra.

    len_x es la longitud de la lista de valores del eje x.
    t_0 es el instante de tiempo (en segundos) en el que se inicia la maniobra.
    '''

    # Crear la lista de valores inicializada en 0
    y_ADS = list(np.zeros(len_x))

    # i es el índice de inicio en el tiempo t_0
    i = int(t_0 * 100)

    # Definir el incremento de la posición por cada paso
    incremento = 0.002

    # Asegurarse de que la maniobra no se salga de los límites de la lista
    max_pasos = min(500, len_x - i)  # 500 pasos corresponden a 5 segundos

    # Realizar la maniobra incrementando el valor
    for paso in range(max_pasos):
        y_ADS[i + paso] = incremento * paso

    # Si llegamos al final de la maniobra, completar con 1 el resto
    for j in range(i + max_pasos, len_x):
        y_ADS[j] = 1

    return y_ADS



In [488]:
# Posiciones del coche A (ADS)
x_A = list(posicion_x_ADS)[0:5]
print(len(x_A))
y_A = obtener_coordenadas_y(len(x_A), 2.56)[0:5]

# Posiciones del coche B (VC)
x_B = list(posicion_x_VC)[0:5]
y_B = list(np.ones(len(posicion_x_ADS)))[0:5]


5


In [489]:
#y_A
#x_B

In [490]:
pip install matplotlib imageio


In [491]:
import matplotlib.pyplot as plt
import imageio


In [492]:
# Crear una lista para guardar las imágenes
imagenes = []

# Crear gráficos para cada paso en el tiempo
for i in range(len(x_A)):
    plt.figure(figsize=(12, 6))
    plt.xlim(0, 226)  # Ajusta los límites según tus datos reales
    plt.ylim(-1, 2)
    plt.title("Ejemplo 1: Carril de aceleración de 200 metros")

    # Establecer el fondo gris
    plt.gca().set_facecolor('gray')

    # Ocultar las etiquetas y ticks del eje y
    plt.yticks([])  # Esto elimina las etiquetas y los ticks

    # Agregar líneas de carretera
    plt.plot([0, 180], [-0.5, -0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([199.5, 250],[0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([180, 200.5],[-0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([0, 200.5],[0.5, 0.5], color='white', linestyle='--', linewidth=1)
    plt.axhline(y=1.5, color='white', linestyle='-', linewidth=1)

    # Graficar posiciones de los coches en el paso actual
    plt.scatter(x_A[i], y_A[i], c='blue', label='ADS', s=500)
    plt.scatter(x_B[i], y_B[i], c='red', label='VC', s=500)

    # Añadir leyenda
    plt.legend(prop={'size': 19})

    # Guardar la figura en un archivo temporal y agregarla a la lista de imágenes
    filename = f'temp_{i}.png'
    plt.savefig(filename)
    imagenes.append(imageio.imread(filename))

    # Cerrar la figura para no sobrecargar memoria
    plt.close()

# Crear un GIF a partir de las imágenes
imageio.mimsave('animacion_coches.gif', imagenes, duration=0.00001)


<ipython-input-492-dacd78a114a8>:34: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imagenes.append(imageio.imread(filename))


In [493]:
# Ahora hagamos lo mismo pero calculemos en el ejemplo 120 metros secuencial

# para ello usemos funciones de secuencial:

t_ini = 0
t_max = 1.99

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 200
agresividad = True

a_a0 = 0
a_h0 = 0


trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                     x_h = x_h0, v_h = v_h0, a_h = a_h0,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)

trayectoria_0_2

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

t_ini = 2
t_max = 20

a_a1 = 0
a_h1 = 1.5

trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                     x_h = x_h1, v_h = v_h1, a_h = a_h1,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)



trayectoria_2_fin

trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))

# Posiciones del coche A (ADS)
x_A = list(trayectoria_0_fin[:,1])
print(len(x_A))
y_A = obtener_coordenadas_y(len(x_A), 4.15)

# Posiciones del coche B (VC)
x_B = list(trayectoria_0_fin[:,4])
y_B = list(np.ones(len(x_A)))


print(len(y_A))

928
928


In [494]:
maniobra_factible_secuencial(trayectoria_0_fin)

,Maniobra,t_inicio_maniobra,x_a,v_a,a_a,x_h,v_h,a_h,dist_coches,bool,Tiempo para la maniobra,Velocidad ADS final maniobra
0,1.0,4.15,97.8,20.0,0.0,102.826875,27.225,1.5,5.026875,1.0,5.13,20.0


In [495]:

# Crear gráficos para cada paso en el tiempo
plt.figure(figsize=(12, 6))
plt.xlim(0, 226)  # Ajusta los límites según tus datos reales
plt.ylim(-1, 2)
plt.title("Ejemplo 1: Carril de aceleración de 200 metros")

# Establecer el fondo gris
plt.gca().set_facecolor('gray')

# Ocultar las etiquetas y ticks del eje y
plt.yticks([])  # Esto elimina las etiquetas y los ticks

# Agregar líneas adicionales
plt.plot([0, 180], [-0.5, -0.5], color='white', linestyle='-', linewidth=1)
plt.plot([199.5, 250],[0.5, 0.5], color='white', linestyle='-', linewidth=1)
plt.plot([180, 200.5],[-0.5, 0.5], color='white', linestyle='-', linewidth=1)
plt.plot([0, 200.5],[0.5, 0.5], color='white', linestyle='--', linewidth=1)
plt.axhline(y=1.5, color='white', linestyle='-', linewidth=1)


for i in range(len(x_A)):
  # Graficar posiciones de los coches en el paso actual
  plt.scatter(x_A[i], y_A[i], c='blue', label='ADS', s=50)
  plt.scatter(x_B[i], y_B[i], c='red', label='VC', s=50)

# Añadir leyenda
#plt.legend(prop={'size': 19})

# Guardar la figura en un archivo temporal y agregarla a la lista de imágenes
filename = f'temp_{i}.png'
plt.savefig(filename)
# Cerrar la figura
plt.close()



In [496]:
# Ahora hagamos lo mismo pero calculemos en el ejemplo 120 metros secuencial

# para ello usemos funciones de secuencial:

t_ini = 0
t_max = 1.99

x_a0 = 15
v_a0 = 20
x_h0 = 0
v_h0 = 24

v_min = 12.5
v_max = 25
x_fin_carril = 120
agresividad = True

a_a0 = -3
a_h0 = 0


trayectoria_0_2 = trayectoria_a_trozos(x_a = x_a0, v_a = v_a0, a_a = a_a0,
                     x_h = x_h0, v_h = v_h0, a_h = a_h0,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)

trayectoria_0_2

(t_ini_1, x_a1, v_a1, a_a1, x_h1, v_h1, a_h1, dist) = trayectoria_0_2[-1]

t_ini = 2
t_max = 20

a_a1 = -3
a_h1 = 0

trayectoria_2_fin = trayectoria_a_trozos(x_a = x_a1, v_a = v_a1, a_a = a_a1,
                     x_h = x_h1, v_h = v_h1, a_h = a_h1,
                     x_final = x_fin_carril,
                     t_ini = t_ini, t_max = t_max,
                     v_min = v_min, v_max = v_max,
                     agresivo = agresividad)



trayectoria_2_fin

trayectoria_0_fin = np.vstack((trayectoria_0_2, trayectoria_2_fin))

# Posiciones del coche A (ADS)
x_A = list(trayectoria_0_fin[:,1])
print(len(x_A))
y_A = obtener_coordenadas_y(len(x_A), 2.57)

# Posiciones del coche B (VC)
x_B = list(trayectoria_0_fin[:,4])
y_B = list(np.ones(len(x_A)))


print(len(y_A))

768
768


In [497]:
## PARA 120 METROS
# Crear gráficos para cada paso en el tiempo
plt.figure(figsize=(12, 6))
plt.xlim(0, 226)  # Ajusta los límites según tus datos reales
plt.ylim(-1, 2)
plt.title("Ejemplo 1: Carril de aceleración de 120 metros")

# Establecer el fondo gris
plt.gca().set_facecolor('gray')

# Ocultar las etiquetas y ticks del eje y
plt.yticks([])  # Esto elimina las etiquetas y los ticks

# Agregar líneas adicionales
plt.plot([0, 100], [-0.5, -0.5], color='white', linestyle='-', linewidth=1)
plt.plot([120, 250],[0.5, 0.5], color='white', linestyle='-', linewidth=1)
plt.plot([100, 120],[-0.5, 0.5], color='white', linestyle='-', linewidth=1)
plt.plot([0, 120],[0.5, 0.5], color='white', linestyle='--', linewidth=1)
plt.axhline(y=1.5, color='white', linestyle='-', linewidth=1)


for i in range(len(x_A)):
  # Graficar posiciones de los coches en el paso actual
  plt.scatter(x_A[i], y_A[i], c='blue', label='ADS', s=50)
  plt.scatter(x_B[i], y_B[i], c='red', label='VC', s=50)

# Añadir leyenda
#plt.legend(prop={'size': 19})

# Guardar la figura en un archivo temporal y agregarla a la lista de imágenes
filename = f'temp_{i}.png'
plt.savefig(filename)
# Cerrar la figura
plt.close()

